In [1]:
from pulp import *
import pandas as pd

# Actividad 1

## Datos

##### Potencias mínimas

In [2]:
P1_min = 150 #[MW]
P2_min = 75 #[MW]
P3_min = 0 #[MW]

##### Potencias máximas

In [3]:
P1_max = 350 #[MW]
P2_max = 150 #[MW]
P3_max = 100 #[MW]

##### Costos fijos

In [4]:
cf1 = 1 #[$/h]
cf2 = 0.5 #[$/h]
cf3 = 0.1 #[$/h]

##### Costos variables

In [5]:
cv1 = 40 #[$/MWh]
cv2 = 75 #[$/MWh]
cv3 = 110 #[$/MWh]

##### Demanda

In [6]:
D = 500

## Problema de optmización primal:
### Minimización de costos

##### Inicialización del problema

In [7]:
prob = LpProblem("Primal", LpMinimize)

##### Definición de variables

In [8]:
P1 = LpVariable("P1")
P2 = LpVariable("P2")
P3 = LpVariable("P3")

##### Función objetivo

In [9]:
prob += (cf1 + cv1 * P1) + (cf2 + cv2 * P2) + (cf3 + cv3 * P3)

##### Restricciones

In [10]:
prob += D - (P1 + P2 + P3) == 0 # lambda
prob += P1_min - P1 <= 0 # m1_min
prob += P1 - P1_max <= 0 # m1_max
prob += P2_min - P2 <= 0 # m2_min
prob += P2 - P2_max <= 0 # m2_max
prob += P3_min - P3 <= 0 # m3_min
prob += P3 - P3_max <= 0 # m3_max

##### Resolución

In [11]:
prob.solve()

1

##### Estado de resolución del problema

In [12]:
print(LpStatus[prob.status])

Optimal


##### Potencias en el punto óptimo

In [13]:
for potencia in prob.variables():
    print(potencia.name , "=", potencia.varValue, "[MW]")

P1 = 350.0 [MW]
P2 = 150.0 [MW]
P3 = 0.0 [MW]


##### Costo total del sistema

In [14]:
print("Costo total = ", value(prob.objective), "[$]")

Costo total =  25251.6 [$]


##### Multiplicadores de Lagrange

In [15]:
for restriccion in prob.constraints.values():
    print("Variable dual para la restricción", restriccion, "->", restriccion.pi)

Variable dual para la restricción -P1 - P2 - P3 = -500 -> -110.0
Variable dual para la restricción -P1 <= -150 -> 0.0
Variable dual para la restricción P1 <= 350 -> -70.0
Variable dual para la restricción -P2 <= -75 -> 0.0
Variable dual para la restricción P2 <= 150 -> -35.0
Variable dual para la restricción -P3 <= 0 -> 0.0
Variable dual para la restricción P3 <= 100 -> 0.0


##### Precios sombra

In [16]:
for P in prob.variables():
    print("Precio sombra de", P, "=", P.dj)

Precio sombra de P1 = 0.0
Precio sombra de P2 = 0.0
Precio sombra de P3 = 0.0


## Problema de optmización dual:
### Maximización de los beneficios

##### Inicialización del problema

In [17]:
prob = LpProblem("Dual", LpMaximize)

##### Definición de variables

In [18]:
l = LpVariable("l")
m1_min = LpVariable("m1_min")
m1_max = LpVariable("m1_max")
m2_min = LpVariable("m2_min")
m2_max = LpVariable("m2_max")
m3_min = LpVariable("m3_min")
m3_max = LpVariable("m3_max")

##### Función objetivo

In [19]:
prob += D * l + (P1_min * m1_min + P2_min * m2_min + P3_min * m3_min) - (P1_max * m1_max + P2_max * m2_max + P3_max * m3_max)

##### Restricciones

In [20]:
prob += (l + m1_min - m1_max) - cv1 <= 0 # P1
prob += (l + m2_min - m2_max) - cv2 <= 0 # P2
prob += (l + m3_min - m3_max) - cv3 <= 0 # P3
prob += -m1_min <= 0
prob += -m1_max <= 0
prob += -m2_min <= 0
prob += -m2_max <= 0
prob += -m3_min <= 0
prob += -m3_max <= 0

##### Resolución

In [21]:
prob.solve()

1

##### Estado de resolución del problema

In [22]:
print(LpStatus[prob.status])

Optimal


##### Multiplicadores de Lagrange en el punto óptimo

In [23]:
for m in prob.variables():
    print(m.name , "=", m.varValue, "[$/MW]")

l = 75.0 [$/MW]
m1_max = 35.0 [$/MW]
m1_min = -0.0 [$/MW]
m2_max = -0.0 [$/MW]
m2_min = -0.0 [$/MW]
m3_max = -0.0 [$/MW]
m3_min = -0.0 [$/MW]


##### Beneficio total del sistema

In [24]:
print("Beneficio total = ", value(prob.objective), "[$]")

Beneficio total =  25250.0 [$]


##### Multiplicadores de Lagrange

In [25]:
for restriccion in prob.constraints.values():
    print("Variable dual para la restricción", restriccion, "->", restriccion.pi)

Variable dual para la restricción l - m1_max + m1_min <= 40 -> 350.0
Variable dual para la restricción l - m2_max + m2_min <= 75 -> 150.0
Variable dual para la restricción l - m3_max + m3_min <= 110 -> -0.0
Variable dual para la restricción -m1_min <= 0 -> 200.0
Variable dual para la restricción -m1_max <= 0 -> -0.0
Variable dual para la restricción -m2_min <= 0 -> 75.0
Variable dual para la restricción -m2_max <= 0 -> 0.0
Variable dual para la restricción -m3_min <= 0 -> -0.0
Variable dual para la restricción -m3_max <= 0 -> 100.0


##### Precios sombra
###### ¿Qué son los precios sombra de las variables duales (Multiplicadores de Lagrange) en el problema dual?

In [26]:
for m in prob.variables():
    print("Precio sombra de", m, "=", m.dj)

Precio sombra de l = 0.0
Precio sombra de m1_max = 0.0
Precio sombra de m1_min = 0.0
Precio sombra de m2_max = 0.0
Precio sombra de m2_min = 0.0
Precio sombra de m3_max = 0.0
Precio sombra de m3_min = 0.0


# Actividad 2

## Datos

### Generadores

In [44]:
Datos_gen = pd.read_excel("Datos_gen.xlsx", sheet_name="Datos_gen", index_col=0)
print(Datos_gen)

            Pmax  Pmin  Coef_a  Coef_b  Coef_c  Cost. comb.  Unnamed: 7   
Gen Number                                                                
1            500   150    2000      25    0.05            1         NaN  \
2            700   200    1250      15    0.04            1         NaN   
3            200     0    1700      10    0.03            1         NaN   

                                    Unnamed: 8  
Gen Number                                      
1           H1 = coef_a + coef_bP1 + coef_cP12  
2                                          NaN  
3                                          NaN  


#### Potencias mínimas

In [48]:
P1_min = Datos_gen.loc[1, "Pmin"] #[MW]
P2_min = Datos_gen.loc[2, "Pmin"] #[MW]
P3_min = Datos_gen.loc[3, "Pmin"] #[MW]
print("P1_min = " + str(P1_min))
print("P2_min = " + str(P2_min))
print("P3_min = " + str(P3_min))

P1_min = 150
P2_min = 200
P3_min = 0


#### Potencias máximas

In [54]:
P1_max = Datos_gen.loc[1, "Pmax"] #[MW]
P2_max = Datos_gen.loc[2, "Pmax"] #[MW]
P3_max = Datos_gen.loc[3, "Pmax"] #[MW]
print("P1_max = " + str(P1_max))
print("P2_max = " + str(P2_max))
print("P3_max = " + str(P3_max))

P1_max = 500
P2_max = 700
P3_max = 200


### Demanda

In [55]:
Perfil_horario = pd.read_excel("Datos_gen.xlsx", sheet_name="Perfil_horario", index_col=0)
print(Perfil_horario)

      Demanda
Hora         
1      836.92
2      805.56
3      786.94
4      781.06
5      783.02
6      794.78
7      847.70
8      903.56
9      951.58
10     995.68
11    1009.40
12    1015.28
13    1015.28
14    1006.46
15    1001.56
16     993.72
17     980.00
18     940.80
19     957.46
20     992.74
21     984.90
22     966.28
23     965.30
24     930.02


### Líneas de transmisión

In [56]:
Transmision = pd.read_excel("Datos_gen.xlsx", sheet_name="Transmision", index_col=0)
print(Transmision)

            B1-B2  B2-B3  B1-B3 Unnamed: 4 Unnamed: 5 Unnamed: 6
Linea                                                           
Capacidad   220.0  280.0    100         MW        NaN        NaN
Reactancia    0.5    0.6      1         pu    100 MVA     220 kV


### Flujo mínimo

In [57]:
P12_min = - Transmision.loc["Capacidad", "B1-B2"]
P23_min = - Transmision.loc["Capacidad", "B2-B3"]
P13_min = - Transmision.loc["Capacidad", "B1-B3"]
print("P12_min = " + str(P12_min))
print("P23_min = " + str(P23_min))
print("P13_min = " + str(P13_min))

P12_min = -220.0
P23_min = -280.0
P13_min = -100


### Flujo máximo

In [58]:
P12_max = - P12_min
P23_max = - P23_min
P13_max = - P13_min
print("P12_max = " + str(P12_max))
print("P23_max = " + str(P23_max))
print("P13_max = " + str(P13_max))

P12_max = 220.0
P23_max = 280.0
P13_max = 100


### Reactancias

In [61]:
Zb = 220 ** 2 / 100
X12 = 0.5 * Zb
X23 = 0.6 * Zb
X13 = 1 * Zb
print("X12 = " + str(X12))
print("X23 = " + str(X23))
print("X13 = " + str(X13))

X12 = 242.0
X23 = 290.4
X13 = 484.0
